<center>

# **如何运行fMRIPrep Package**

</center>

***

本教程将教你如何在 Windows、macOS 和 Linux 系统 上使用 fMRIPrep 对 BIDS 格式的 fMRI 数据进行标准化预处理。

我们将使用 Docker 运行 fMRIPrep，以确保跨平台兼容和最大程度的可重复性。

- **Step 1: 准备 BIDS 格式的数据集**
- **Step 2: 通过 Docker 运行fMRIPrep**
- **Step 3: FreeSurfer 许可证**
- **Step 4: 运行 fMRIPrep**
- **Step 5: 检查输出结果**



# Step 1 准备 BIDS 格式的数据集

### 输入数据：符合BIDS格式的数据集


```
Bids/
├── dataset_description.json
├── participants.tsv
└── sub-001/
    ├── anat/
    │   ├── sub-001_T1w.nii.gz
    │   └── sub-001_T1w.json
    └── func/
        ├── sub-001_task-rest_bold.nii.gz
        └── sub-001_task-rest_bold.json
```


# Step 2 通过 Docker 运行fMRIPrep

In [ ]:
# 下列命令在linux、macos、windows下均可执行
docker pull nipreps/fmriprep:latest

**注意：配置 Docker 镜像加速器**

在国内拉取 Docker 镜像时，如果出现如下错误：

```
docker: Error response from daemon: Get "https://registry-1.docker.io/v2/": read tcp 192.168.30.91:51536->44.208.254.194:443: read: connection reset by peer.
```

这表明 Docker 客户端在尝试访问 Docker Hub 或其他注册中心时出错，具体是无法获取注册中心的 API 响应。

> 🔗 参考：[知乎文章 - Docker 镜像加速](https://zhuanlan.zhihu.com/p/24228872523)


打开 Docker Desktop → Settings → Docker Engine，添加以下内容：

```json
{
  "registry-mirrors": [
    "https://docker-cf.registry.cyou",
    "https://mirror.aliyuncs.com",
    "https://mirror.baidubce.com",
    "https://docker.nju.edu.cn",
    "https://docker.mirrors.ustc.edu.cn",
    "https://mirrors.tuna.tsinghua.edu.cn"
  ]
}
```

点击 Apply & Restar。

之后在命令行/终端再次输入docker pull 的相关命令。

# Step 3: 运行 fMRIPrep 的被试级预处理分析

**注意：**

fMRIPrep 使用 FreeSurfer 来重建大脑的解剖表面并执行部分配准步骤，因此需要 FreeSurfer 授权许可。

获取许可的方法如下：

1. 免费注册并申请许可：[FreeSurfer License 申请页面](https://surfer.nmr.mgh.harvard.edu/fswiki/License)
2. 填写注册表单后，许可文件（`license.txt`）将会发送到你填写的邮箱。
3. 下载并妥善保存该许可文件，后续运行 fMRIPrep 时需要用到。

当你获得 FreeSurfer 许可之后，就可以使用 Docker 运行 fMRIPrep 了。

下面是一个示例 Python 脚本（可在 Linux、macOS、Windows 上运行）：

In [ ]:
import os

# 需要处理的被试编号列表
subs = ['014', '015', '017']

# BIDS 数据集根目录
bids_root_dir = "/mnt/sdb1/Judge-fMRI-Data/Bids"

# fMRIPrep 输出目录
bids_out_dir = "/mnt/sdb1/Judge-fMRI/fmriprep"

# FreeSurfer 授权文件路径
license_path = "/mnt/sdb1/Judge-fMRI/Scripts/ProcessingScripts/fMRIPrep/license.txt"

# 线程数
nthreads = 12

# 检查 fMRIPrep 版本
print("Checking fMRIPrep version...")
os.system("docker run --rm -it nipreps/fmriprep --version")

# 循环处理每个被试
for sub in subs:
    # 构建 docker 命令
    cmd = f"""
    docker run --rm -it \
        -v {bids_root_dir}:/inputbids \
        -v {bids_out_dir}/fmriprep/sub-{sub}:/output \
        -v {license_path}:/license.txt \
        nipreps/fmriprep \
        /inputbids /output participant \
        --participant_label {sub} \
        --nthreads {nthreads} --omp-nthreads {nthreads} \
        --mem-mb 32000 --ignore slicetiming --ignore fieldmaps \
        --fs-license-file /license.txt \
        --verbose
    """
    print(f"Running fMRIPrep for sub-{sub} ...")
    os.system(cmd)

# Step 4: 检查输出结果和日志文件

运行完 fMRIPrep 后，建议打开生成的可视化html报告，可以在浏览器中查看（推荐使用 Chrome 或 Firefox）。

报告将引导你了解预处理步骤，并突出展示 fMRIPrep 过程中所做的一些关键决策。

建议仔细检查每位被试的报告，以评估预处理结果的质量。

在每个报告的“Methods”章节，你会找到对具体处理流程的详细描述，方便直接复制到你的论文中，并包含所有相关参考文献。

fMRIPrep 完成后，输出将位于子目录中 `derivatives/fmriprep` 。

在该目录中，我们可以看到每个被试都会有一个独立的文件夹以及每个被试的 HTML报告，如 `sub-001` 文件夹和对应的 `sub-001.html` ：

<div align="center">
  <img src="https://cdn.kesci.com/upload/sz6chowgo.png?imageView2/0/w/1280/h/1280" width="400"/>
</div>

现在，我们打开 `sub-001` 文件夹， 我们可以看到每个被试都包含了所有预处理的功能 (func) 和解剖 (anat) 数据，包括包含预处理中各部分结果图片的 `figure` 文件夹，但这些图片结果在 `.html` 中都有报告，所以不在我们本次的解释范围内，我们重点讲解  `anat` 和  `func` 两个文件夹。

首先，我们查看 `anat` 文件夹结构：

<div align="center">
  <img src="https://cdn.kesci.com/upload/sz6csu2nne.png?imageView2/0/w/1280/h/1280" width="500"/>
</div>

其次，我们查看 `func` 文件夹结构：


<div align="center">
  <img src="https://cdn.kesci.com/upload/sz6ct6ntp.png?imageView2/0/w/1280/h/1280" width="500"/>
</div>
